In [1]:
path_data = '/data'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd

import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

plt.style.use('ggplot')

Torch version: 2.2.1+cu121


In [3]:
data_unfiltered = pd.read_csv("/".join([path_data, "images_v2.csv"]))
#display(data_unfiltered)

In [4]:
len(data_unfiltered)

38479

### Remove subjective topics

In [5]:
subjective_topics = ['Favorite home decorations', 'Favourite item in kitchen', 'Favourite sports clubs', 'How the most loved item is used', 'icons', 'Idols', 'Latest furniture bought', ' looking over the shoulder', 'Most loved item', 'Most loved toy', 'Most played songs on the radio', 'Music idol', 'Next big thing you are planning to buy', 'Playing with most loved toy', 'Thing I dream about having', 'Things I wish I had', 'Using most loved item', 'Youth culture', 'What I wish I could buy']

In [6]:
len(subjective_topics)

19

In [7]:
unique_topics = data_unfiltered['topics'].unique()

In [8]:
Subjective_topic_list = []

for topic in unique_topics:
    topic_ = topic.replace('/ ', ', ')
    topic_split = topic_.split(',')
    for split in topic_split:
        if split in subjective_topics:
            Subjective_topic_list.append(topic)

In [9]:
data = data_unfiltered[~data_unfiltered['topics'].isin(Subjective_topic_list)]
display(data)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
5,5d4bf31dcf0b3a0f3f35983c,Burundi,bi,af,image,assets/5d4bf31dcf0b3a0f3f35983c/5d4bf31dcf0b3a...,Plate of food,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


In [10]:
len(data)

36754

### Split topics

In [11]:
from collections import Counter
list_topics = list(data['topics'])


separate_topics = [t.lower().strip() for topic in list_topics for t in topic.split(",") ]


print(len(separate_topics))
#print(Counter(separate_topics))
set_topics = list(set(separate_topics))
print(len(set_topics))

45692
270


In [12]:
len(separate_topics)

45692

In [13]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set() #### here a set was used in place of list to avoid duplicate where keyword appears twice in a topic
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

270


In [14]:
#sanity checks 

ground_truth_counts = [len(dict_topic2img[i]) for i in dict_topic2img]
print(sum(ground_truth_counts))

45616


In [15]:
topics = dict_topic2img.keys()

### Get ground truth

In [16]:
temp = "This is a photo of "
English_topics = [temp + topic for topic in topics]

In [17]:
topic2english = {}
for topic, prompt in zip(topics, English_topics):
    topic2english[topic] = prompt

In [18]:
#topic2english

In [19]:
english_dict_topic2imgGT = {}

for topic in topics:
    key = topic2english[topic]
    english_dict_topic2imgGT[key] = dict_topic2img[topic]
print(len(english_dict_topic2imgGT))

270


In [20]:
len(english_dict_topic2imgGT['This is a photo of spices'])

419

In [21]:
#income_grps = ['a poor country', 'a rich country']


income_grps = ['a poor country', 'a low income country', 'a cheap country', 'an impoverished country', 'a rich country', 'a high income country', 'an expensive country', 'a wealthy country', 'a country', 'a poor region', 'a low income region', 'a cheap region', 'an impoverished region', 'a rich region', ' a high income region', 'an expensive region', 'a wealthy region', 'a region', 'a poor place', 'a low income place', 'a cheap place', 'an impoverished place', 'a rich place', 'a high income place', 'an expensive place', 'a wealthy place', 'a place', 'a poor home',  'a low income home', 'a cheap home', 'an impoverished home', 'a rich home', 'a high income home', 'an expensive home', 'a wealthy home', 'a home']

In [22]:
list_of_income_dicts = []  # list of ground truth imgs and text prompt mapping for each country; list of dict_topic2img for each country
list_of_topic2prompt_dict = []  # list of text prompt and topic mapping

for income in income_grps:
    
    income_topics = ['This is a photo of ' + topic + ' from ' + income for topic in topics]
      
    topic2income = {}
    for topic, prompt in zip(topics, income_topics):
        topic2income[topic] = prompt
    list_of_topic2prompt_dict.append(topic2income)
    
    
    dict_incometopic2imgGT = {}

    for topic in topics:
        key = topic2income[topic]
        dict_incometopic2imgGT[key] = dict_topic2img[topic]
        
    list_of_income_dicts.append(dict_incometopic2imgGT)

In [23]:
len(list_of_income_dicts)

36

## Plotting results

In [24]:
import pickle
with open("data/income_list_dict_topic2FNimg.pkl", 'rb') as f:
    lang_list_dict_topic2FNimg = pickle.load(f)

In [25]:
with open("data/income_list_dict_topic2F1.pkl", 'rb') as f:
    lang_list_dict_topic2F1 = pickle.load(f)

In [26]:
labels=["poor", "lower-middle", "upper-middle", "rich"]

#list_of_topic2prompt_dict[1]
data["quartile"] = pd.qcut(data["income"], q=[0, 0.25, 0.5, 0.75, 1],    
                         labels=labels)

In [27]:
# map each image to corresponding topics, countries, income level ... TODO
dict_img2topic, dict_img2country, dict_img2incomelevel, dict_country2region = {}, {}, {}, {}
for list_topics, image_path, country, incomelevel, region in zip(data['topics'], data['imageRelPath'], data['country.name'], data['quartile'], data['region.id']):
    dict_img2topic[image_path] = list_topics
    dict_img2country[image_path] = country
    dict_img2incomelevel[image_path] = incomelevel
    dict_country2region[country] = region

print(len(dict_img2topic))
# print(dict_img2country)

36754


## Automating the analysis across all prompts

In [35]:
from collections import Counter

all_topic_all_income_percountrydf = pd.DataFrame()
all_topic_all_country_perincomedf = pd.DataFrame()
language_recall_df = pd.DataFrame()
for lang_dict in tqdm.tqdm(lang_list_dict_topic2FNimg):
    
    #### forgotten images block (countrywise)
    
    list_all_countriesFN = []
    list_all_incomeFN = []
    dict_topic2FNCountry = {}
    dict_all_Countries_FN = {}
    
    for topic in lang_dict:
        if topic.split(' ')[-2] =='a':
            country_suffix = topic.split(' ')[-2:]
        elif topic.split(' ')[-2] == 'income':
            country_suffix = topic.split(' ')[-4:]
        else:
            country_suffix = topic.split(' ')[-3:]
        country_suffix = ' '.join(country_suffix)
        #dict_lang2translation = next(item for item in list_dict_lang2translation if topic.split(';')[1].strip() in list(item.values())[0])
        #dominant_lang = list(dict_lang2translation.keys())[0]
        list_imagesFN = lang_dict[topic]
        list_countriesFN = [dict_img2country[img] for img in list_imagesFN]
        list_incomepercountry = [dict_img2incomelevel[img] for img in list_imagesFN]
        
        for country in list_countriesFN:
            list_all_countriesFN.append(country)
        for income in list_incomepercountry:
            list_all_incomeFN.append(income)
        
        dict_topic2FNCountry[topic] = Counter(list_countriesFN)

    dict_all_Countries_FN = Counter(list_all_countriesFN)
    
    
    #### ground truth images block (countrywise)
    
    list_all_countriesGT = []
    list_all_incomeGT = []
    dict_topic2GTCountry = {}
    dict_all_Countries_GT = {}
    
    dict_lang_topic2img = next(item for item in list_of_income_dicts if topic in item.keys())
    
    for topic in dict_lang_topic2img:
        list_imagesGT = dict_lang_topic2img[topic]
        list_countriesGT = [dict_img2country[img] for img in list_imagesGT]
        list_incomepercountryGT = [dict_img2incomelevel[img] for img in list_imagesGT]
        for country in list_countriesGT:
            list_all_countriesGT.append(country)
        for income in list_incomepercountryGT:
            list_all_incomeGT.append(income)
            
        dict_topic2GTCountry[topic] = Counter(list_countriesGT)

    
    dict_all_Countries_GT = Counter(list_all_countriesGT)
    
    
    
    ### forgotten images block (income level wise)
    
    list_all_incomelevelsFN = []
    dict_topic2FNIncomelevels = {}
    dict_all_incomelevels_FN = {}
    
    mapped_dict_lang_topic2FNimg = next(item for item in lang_list_dict_topic2FNimg if topic in item.keys())
    
    for topic in mapped_dict_lang_topic2FNimg:
        list_imagesFN = mapped_dict_lang_topic2FNimg[topic]
        list_incomelevelsFN = [dict_img2incomelevel[img] for img in list_imagesFN]
        for incomelevel in list_incomelevelsFN:
            list_all_incomelevelsFN.append(incomelevel)

        dict_topic2FNIncomelevels[topic] = Counter(list_incomelevelsFN)


    dict_all_incomelevels_FN = Counter(list_all_incomelevelsFN)
    
    
    
    ### ground truth image block (income level wise)
    
    ### Income level ground truth

    list_all_incomelevelsGT = []
    dict_topic2GTIncomelevels = {}
    dict_all_incomelevels_GT = {}
    
    mapped_dict_lang_topic2img = next(item for item in list_of_income_dicts if topic in item.keys())
    for topic in mapped_dict_lang_topic2img:
        list_imagesGT = mapped_dict_lang_topic2img[topic]
        list_incomelevelsGT = [dict_img2incomelevel[img] for img in list_imagesGT]
        for incomelevel in list_incomelevelsGT: 
            list_all_incomelevelsGT.append(incomelevel)
        
        dict_topic2GTIncomelevels[topic] = Counter(list_incomelevelsGT)

    
    dict_all_incomelevels_GT = Counter(list_all_incomelevelsGT)

    
    #### dataframe building
    
    ################### all_topic_all_income_percountrydf ###########
    
    Forget_ratio_allincome_percountry = Counter({key : dict_all_Countries_FN[key] / dict_all_Countries_GT[key] for key in dict_all_Countries_FN})
    Forget_ratio_allincome_percountry_df = pd.DataFrame(Forget_ratio_allincome_percountry.items(),
                        columns = ['Country', 'Forget ratio'])
    Forget_ratio_allincome_percountry_df['country_suffix'] = [country_suffix for i in list(Forget_ratio_allincome_percountry_df['Forget ratio'])]
    Forget_ratio_allincome_percountry_df['Total images per country'] = Forget_ratio_allincome_percountry_df['Country'].map(dict_all_Countries_GT)
    Forget_ratio_allincome_percountry_df['Forgotten images'] = Forget_ratio_allincome_percountry_df['Country'].map(dict_all_Countries_FN)
    Forget_ratio_allincome_percountry_df['Recognized images'] = Forget_ratio_allincome_percountry_df['Total images per country'] - Forget_ratio_allincome_percountry_df['Forgotten images']
    Forget_ratio_allincome_percountry_df['Recall'] = Forget_ratio_allincome_percountry_df['Recognized images'] / Forget_ratio_allincome_percountry_df['Total images per country']
    
    all_topic_all_income_percountrydf = pd.concat([all_topic_all_income_percountrydf, Forget_ratio_allincome_percountry_df])
    
    
    ####################### alltopic_all_country_perincome ###################
    
    Forget_ratio_all_country_perincome = Counter({key : dict_all_incomelevels_FN[key] / dict_all_incomelevels_GT[key] for key in dict_all_incomelevels_FN})
    Forget_ratio_all_country_perincome_df = pd.DataFrame(Forget_ratio_all_country_perincome.items(), columns = ['Income level', 'Forget ratio',])
    Forget_ratio_all_country_perincome_df['Total images per income level'] = Forget_ratio_all_country_perincome_df['Income level'].map(dict_all_incomelevels_GT)
    Forget_ratio_all_country_perincome_df['Forgotten images'] = Forget_ratio_all_country_perincome_df['Income level'].map(dict_all_incomelevels_FN)
    #Forget_rationperincome_allcountry_df = Forget_ratio_percountry_allincome_df
    Forget_ratio_all_country_perincome_df['country_suffix'] = [country_suffix for i in list(Forget_ratio_all_country_perincome_df['Forget ratio'])]
    Forget_ratio_all_country_perincome_df['Recognized images'] = Forget_ratio_all_country_perincome_df['Total images per income level'] - Forget_ratio_all_country_perincome_df['Forgotten images']
    Forget_ratio_all_country_perincome_df['Recall'] = Forget_ratio_all_country_perincome_df['Recognized images'] / Forget_ratio_all_country_perincome_df['Total images per income level']
    
    all_topic_all_country_perincomedf = pd.concat([all_topic_all_country_perincomedf, Forget_ratio_all_country_perincome_df])
    
    
    
    ######################## all topic, per country, per income #####################
    
    Lang_FN_df = pd.DataFrame()
    Lang_FN_df['countries'] = list_all_countriesFN
    Lang_FN_df['country_suffix'] = [country_suffix for i in list_all_countriesFN]
    Lang_FN_df['income level'] = list_all_incomeFN
    Lang_forgotten = Lang_FN_df.groupby(['countries', 'income level']).count()
    Lang_forgotten.reset_index()
    
    lang_countries = [x[0] for x in Lang_forgotten.index]
    lang_incomelevel = [x[1] for x in Lang_forgotten.index]
    
    Lang_GT_df = pd.DataFrame()
    Lang_GT_df['countries'] = list_all_countriesGT
    Lang_GT_df['country_suffix'] = [country_suffix for i in list_all_countriesGT]
    Lang_GT_df['income level'] = list_all_incomeGT

    
    Lang_GT = Lang_GT_df.groupby(['countries', 'income level']).count()
    Lang_GT.reset_index()
    
    Lang_recall_df = pd.DataFrame()

    Lang_recall_df['country_suffix'] = [country_suffix for i in lang_countries]
    Lang_recall_df['countries'] = lang_countries
    Lang_recall_df['incomelevel'] = lang_incomelevel
    Lang_recall_df['forgotten'] = list(Lang_forgotten['country_suffix'])
    Lang_recall_df['ground truth'] =  list(Lang_GT['country_suffix'])
    Lang_recall_df['recall'] = 1 - (Lang_recall_df['forgotten']/ Lang_recall_df['ground truth'])
    
    language_recall_df = pd.concat([language_recall_df, Lang_recall_df])
    
    

all_topic_all_income_percountrydf.to_csv('country_all_topic_all_income_percountrydf')
all_topic_all_country_perincomedf.to_csv('country_all_topic_all_country_perincomedf')
language_recall_df.to_csv('country_recall_df')

100%|██████████| 36/36 [00:04<00:00,  8.43it/s]


In [11]:
import pandas as pd
country_all_topic_all_income_percountrydf = pd.read_csv('country_all_topic_all_income_percountrydf')
country_all_topic_all_country_perincomedf = pd.read_csv('country_all_topic_all_country_perincomedf')
country_recall_df = pd.read_csv('country_recall_df')

In [37]:
country_all_topic_all_country_perincomedf['country_suffix'][0]

'a poor country'

In [31]:
poor_suffix = ['a poor country', 'a low income country', 'a cheap country', 'an impoverished country', 'a poor region', 'a low income region', 'a cheap region', 'an impoverished region', 'a poor place', 'a low income place', 'a cheap place', 'an impoverished place', 'a poor home',  'a low income home', 'a cheap home', 'an impoverished home']
rich_suffix = ['a rich country', 'a high income country', 'an expensive country', 'a wealthy country', 'a rich region', 'a high income region', 'an expensive region', 'a wealthy region', 'a rich place', 'a high income place', 'an expensive place', 'a wealthy place', 'a rich home', 'a high income home', 'an expensive home', 'a wealthy home']
neutral_suffix = ['a country', 'a region', 'a place', 'a home']

In [99]:
countries_with_poor = ['Burkina Faso', 'Burundi', 'Liberia', 'Malawi', 'Rwanda', 'Somalia', 'Togo', 'Bangladesh', 'Bolivia', 'Cambodia', 'Cameroon', "Cote d'Ivoire", 'Haiti', 'India', 'Kenya', 'Myanmar', 'Nepal', 'Nigeria', 'Pakistan', 'Palestine', 'Papua New Guinea','Philippines', 'Tanzania', 'Tunisia', 'Zimbabwe', 'China', 'Colombia', 'Indonesia', 'Peru', 'South Africa', 'Thailand' ]
countries_with_low_mid = ['Ethiopia', 'Rwanda', 'Togo', 'Bolivia', 'Cambodia', 'Cameroon', 'Ghana','India', 'Jordan', 'Kenya', 'Mongolia', 'Myanmar', 'Nepal', 'Nigeria', 'Pakistan', 'Palestine', 'Philippines', 'Tanzania', 'Tunisia', 'Ukraine', 'Vietnam', 'Brazil', 'China', 'Colombia', 'Guatemala', 'Indonesia', 'Peru', 'Serbia', 'South Africa', 'Thailand', 'South Korea', 'United States']
countries_with_up_mid = ['Ethiopia','Cambodia', 'Cameroon', 'Egypt', 'India', 'Iran', 'Kyrgyzstan', 'Lebanon', 'Nepal', 'Nigeria', 'Pakistan', 'Philippines', 'Sri Lanka', 'Tanzania', 'Ukraine', 'Brazil', 'China', 'Colombia', 'Indonesia', 'Kazakhstan', 'Mexico', 'Russia', 'Serbia', 'South Africa', 'Thailand', 'Austria', 'France', 'Netherlands', 'South Korea', 'Sweden', 'United Kingdom', 'United States']
countries_with_rich = ['Ethiopia','India', 'Iran', 'Jordan', 'Kenya', 'Nepal', 'Nigeria', 'Pakistan', 'Ukraine', 'Vietnam', 'Brazil', 'China', 'Colombia', 'Indonesia', 'Mexico', 'Russia', 'Serbia', 'South Africa', 'Turkey', 'Austria', 'Canada', 'Czech Republic', 'Denmark', 'France', 'Italy', 'Netherlands', 'Romania', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']
len(countries_with_rich)

33

In [94]:
#income = 'poor'
#income = 'lower-middle'
#income = 'upper-middle'
income = 'rich'

In [100]:
income_country_dict = {}
#per_country = []
poor_df = country_recall_df[country_recall_df['incomelevel'] == income]

countries_with_income_level = poor_df['countries'].unique()

for country in countries_with_rich:
#for country in countries_with_income_level:
    per_country = []
    poor_country = poor_df[poor_df['countries'] == country]
    poor = []
    for i in poor_suffix:
        recall = poor_country[poor_country['country_suffix'] == i]['recall']
        poor.append(recall.item())
    per_country.append(round(sum(poor)/len(poor), 3))
    
    rich = []
    for i in rich_suffix:
        recall = poor_country[poor_country['country_suffix'] == i]['recall']
        rich.append(recall.item())
    per_country.append(round(sum(rich)/len(rich), 3))
    
    neutral = []
    for i in neutral_suffix:
        recall = poor_country[poor_country['country_suffix'] == i]['recall']
        neutral.append(recall.item())
    per_country.append(round(sum(neutral)/len(neutral), 3))
    
    income_country_dict[country] = per_country

In [101]:
#country
len(income_country_dict.keys())

33

In [105]:
for i in income_country_dict.values():
    print(i[2])

0.424
0.365
0.335
0.335
0.532
0.45
0.313
0.333
0.387
0.416
0.288
0.313
0.329
0.486
0.394
0.47
0.354
0.3
0.313
0.217
0.625
0.337
0.245
0.327
0.348
0.372
0.401
0.47
0.329
0.278
0.271
0.408
0.486


In [74]:
# poor_country = poor_df[poor_df['countries'] == 'Bangladesh']
# poor = []
# for i in rich_suffix:
#     recall = poor_country[poor_country['country_suffix'] == i]['recall']
#     poor.append(recall.item())
# round(sum(poor)/len(poor), 3)

In [ ]:
## Query for per income prompt recall for each country for each income group

Country_prompt_recall_list = []

unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    specific = country_all_topic_all_income_percountrydf[country_all_topic_all_income_percountrydf['Country'] == country]
    recall = specific[specific['country_suffix'] == 'a home']['Recall']
    recall = round(recall.item(), 3)
    Country_prompt_recall_list.append(recall)


In [38]:
country_recall_df

,Unnamed: 0,country_suffix,countries,incomelevel,forgotten,ground truth,recall
0,0,a poor country,Austria,rich,251,284,0.116197
1,1,a poor country,Austria,upper-middle,118,144,0.180556
2,2,a poor country,Bangladesh,poor,431,753,0.427623
3,3,a poor country,Bolivia,lower-middle,156,245,0.363265
4,4,a poor country,Bolivia,poor,67,95,0.294737
...,...,...,...,...,...,...,...
4603,123,a home,United States,rich,503,1048,0.520038
4604,124,a home,United States,upper-middle,471,846,0.443262
4605,125,a home,Vietnam,lower-middle,332,491,0.323829
4606,126,a home,Vietnam,rich,72,145,0.503448


In [39]:
## Additional dataframe for lower levels only 

low_lvl_language_recall_df = pd.DataFrame()

unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    all_lvl = country_recall_df[country_recall_df['countries'] == country]
    unique_lvls = list(all_lvl['incomelevel'].unique())
    lvls = []
    for lvl in unique_lvls:
        if lvl in ['poor', 'lower-middle']:
            lvls.append(lvl)

    if len(lvls) == 2:

        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == 'poor']
        lower_lvl_low_mid = all_lvl[all_lvl['incomelevel'] == 'lower-middle']

        forgotten = list(lower_lvl_poor['forgotten']) + (lower_lvl_low_mid['forgotten'])
        ground_truth = list(lower_lvl_poor['ground truth']) + (lower_lvl_low_mid['ground truth'])
        recall = (ground_truth - forgotten) / ground_truth
        df = lower_lvl_poor.drop(['forgotten', 'incomelevel', 'recall', 'ground truth'], axis=1)
        df['incomelevel'] = ['poor, lower-middle' for i in df['countries']]
        df['forgotten'] = list(forgotten)
        df['ground truth'] = list(ground_truth)
        df['recall'] = list(recall)
        low_lvl_language_recall_df = pd.concat([low_lvl_language_recall_df, df])

    elif len(lvls) == 1:
        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == lvls[0]]
        df = lower_lvl_poor
        low_lvl_language_recall_df = pd.concat([low_lvl_language_recall_df, df])

    elif len(lvls) == 0:
        df = pd.DataFrame()

low_lvl_language_recall_df.to_csv('country_low_lvl_recall_df')    

In [40]:
low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == 'Burundi']

,Unnamed: 0,country_suffix,countries,incomelevel,forgotten,ground truth,recall
9,9,a poor country,Burundi,poor,204,331,0.383686
137,9,a low income country,Burundi,poor,210,331,0.365559
265,9,a cheap country,Burundi,poor,244,331,0.262840
393,9,an impoverished country,Burundi,poor,207,331,0.374622
521,9,a rich country,Burundi,poor,208,331,0.371601
649,9,a high income country,Burundi,poor,217,331,0.344411
777,9,an expensive country,Burundi,poor,242,331,0.268882
905,9,a wealthy country,Burundi,poor,223,331,0.326284
1033,9,a country,Burundi,poor,229,331,0.308157
1161,9,a poor region,Burundi,poor,239,331,0.277946


In [43]:
## Additional dataframe for all levels per country_suffix

avg_all_lvl_language_recall_df = pd.DataFrame()

df_countries = list(low_lvl_language_recall_df['countries'].unique())
df_countries.sort()

suffix = []
countri = []
recal = []
income = []

for country in df_countries:
    all_lvl = country_recall_df[country_recall_df['countries'] == country]
    
    for country_suff in df_countries:
        forgotten = []
        ground_truth = []
        #recall = []
        all_lvl_suff = all_lvl[all_lvl['country_suffix']==country_suff]
        
        recall = (all_lvl_suff.sum()['ground truth'] - all_lvl_suff.sum()['forgotten'])/ all_lvl_suff.sum()['ground truth']

        
        suffix.append(country_suff)
        countri.append(country)
        recal.append(recall)
        income.append('all')
           
avg_all_lvl_language_recall_df['country suffix'] = suffix
avg_all_lvl_language_recall_df['country'] = countri
avg_all_lvl_language_recall_df['recall'] = recal

avg_all_lvl_language_recall_df.to_csv('country_all_lvl_recall_df.csv') 

In [44]:
country_all_topic_all_income_percountrydf

,Unnamed: 0,Country,Forget ratio,country_suffix,Total images per country,Forgotten images,Recognized images,Recall
0,0,Burundi,0.616314,poor country,331,204,127,0.383686
1,1,India,0.734578,poor country,5625,4132,1493,0.265422
2,2,Somalia,0.703704,poor country,54,38,16,0.296296
3,3,Papua New Guinea,0.676768,poor country,99,67,32,0.323232
4,4,Rwanda,0.574830,poor country,294,169,125,0.425170
...,...,...,...,...,...,...,...,...
121,58,Sweden,0.852679,rich country,224,191,33,0.147321
122,59,Canada,0.520833,rich country,48,25,23,0.479167
123,60,Denmark,0.810897,rich country,312,253,59,0.189103
124,61,Togo,0.676923,rich country,390,264,126,0.323077


In [47]:
specific = country_recall_df[country_recall_df['countries'] == 'Colombia']
specific[specific['country_suffix'] == 'poor country']


,Unnamed: 0,country_suffix,countries,incomelevel,forgotten,ground truth,recall
21,21,poor country,Colombia,lower-middle,249,322,0.226708
22,22,poor country,Colombia,poor,206,338,0.390533
23,23,poor country,Colombia,rich,578,721,0.198336
24,24,poor country,Colombia,upper-middle,1261,1619,0.221124


In [46]:
country_all_topic_all_income_percountrydf[country_all_topic_all_income_percountrydf['Country'] == 'Burkina Faso'].sort_values('Recall')[-12:]

,Unnamed: 0,Country,Forget ratio,country_suffix,Total images per country,Forgotten images,Recognized images,Recall
99,36,Burkina Faso,0.736842,rich country,589,434,155,0.263158
36,36,Burkina Faso,0.668930,poor country,589,394,195,0.331070


In [49]:
specific = country_all_topic_all_income_percountrydf[country_all_topic_all_income_percountrydf['Country'] == 'Burkina Faso']
recall = specific[specific['country_suffix'] == 'poor country']['Recall']
recall

36    0.33107
Name: Recall, dtype: float64

## Query for table filling

In [42]:
selected_countries = ['Austria', 'Bolivia', 'Brazil', 'Burkina Faso', 'Burundi', 'Cameroon', 'Canada', 'China', 'Colombia', "Cote d'Ivoire", 'Czech Republic', 'Denmark', 'Egypt', 'France', 'Guatemala', 'Haiti', 'India', 'Indonesia', 'Iran', 'Italy', 'Jordan', 'Kazakhstan', 'Lebanon', 'Mexico', 'Mongolia', 'Myanmar', 'Netherlands', 'Pakistan', 'Palestine', 'Peru', 'Romania', 'Russia', 'Serbia', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Thailand', 'Togo', 'Tunisia', 'Turkey', 'Ukraine', 'Vietnam']

In [43]:
df_countries = list(low_lvl_language_recall_df['countries'].unique())
df_countries.sort()

lang_countries = [country for country in selected_countries if country in df_countries]
len(lang_countries)

25

In [193]:
## Query for country prompt recall for each country

Country_prompt_recall_list = []

unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    specific = country_all_topic_all_income_percountrydf[country_all_topic_all_income_percountrydf['Country'] == country]
    recall = specific[specific['country_suffix'] == 'a home']['Recall']
    recall = round(recall.item(), 3)
    Country_prompt_recall_list.append(recall)


In [195]:
## Query for country prompt recall for each country for lower income

Country_prompt_recall_list = []
country_prompt_dict = {}


unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

df_countries = list(low_lvl_language_recall_df['countries'].unique())
df_countries.sort()


for country in unique_countries:
    if country in df_countries:
        specific = low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]
        recall = specific[specific['country_suffix'] == 'a home']['recall']
        recall = round(recall.item(), 3)
        Country_prompt_recall_list.append(recall)
        country_prompt_dict[country] = recall

    else: 
        recall = '-'
        Country_prompt_recall_list.append(recall)
        country_prompt_dict[country] = recall
        

In [196]:
for i in Country_prompt_recall_list:
    print(i)

-
0.202
0.274
0.341
0.083
0.136
0.309
0.271
-
0.264
0.295
0.143
-
-
-
0.278
-
0.321
0.348
0.132
0.246
0.331
-
-
0.385
-
0.281
-
-
0.244
0.096
-
0.361
0.232
0.247
-
0.195
0.262
0.343
0.162
0.224
0.271
-
-
0.289
0.337
0.074
0.22
0.338
-
-
-
-
0.26
0.25
0.236
0.23
-
0.381
-
0.4
0.324
0.096


In [15]:
# ## Query for country prompt recall for each country for low income only

# country_prompt_top_dict = {}
# country_prompt_worst_dict = {}
# list_best_prompt = []
# list_worst_prompt = []
# bestest_prompt = []
# rank_list = []

# unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
# unique_countries.sort()

# for country in unique_countries:
#     if country in df_countries:
    
#         specific = low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]
#         rank = list(specific.sort_values('recall', ascending = False)['country_suffix']).index(country)
#         best_country_prompt = list(specific.sort_values('recall', ascending = False)['country_suffix'][:5])
#         best_country_recall = list(round(specific.sort_values('recall', ascending = False)['recall'][:1], 3))
#         worst_country_prompt = list(specific.sort_values('recall', ascending = False)['country_suffix'][-5:])
#         list_best_prompt.append(best_country_prompt)
#         list_worst_prompt.append(worst_country_prompt[::-1])
#         rank_list.append(rank+1)

#         bestest_country_prompt = best_country_prompt[0], best_country_recall
#         bestest_prompt.append(bestest_country_prompt) 
#         country_prompt_top_dict[country] = best_country_prompt

#         country_prompt_worst_dict[country] = worst_country_prompt[::-1]
        
#     else:
#         list_best_prompt.append(['-'])
#         list_worst_prompt.append(['-'])
#         rank_list.append('-')
        
#         best_lang_prompt = '-'
#         #bestest_lang_prompt = '-', '-'
#         bestest_prompt.append('-') 
#         country_prompt_top_dict[country] = best_lang_prompt

#         country_prompt_worst_dict[country] = '-'
        

In [62]:
#rank_list

In [17]:
# ## Query for country prompt recall for each country

# country_prompt_top_dict = {}
# country_prompt_worst_dict = {}
# list_best_prompt = []
# list_worst_prompt = []
# bestest_prompt = []
# rank_list = []

# unique_countries = list(country_all_topic_all_income_percountrydf['Country'].unique())
# unique_countries.sort()

# for country in unique_countries:
    
#     specific = country_all_topic_all_income_percountrydf[country_all_topic_all_income_percountrydf['Country'] == country]
#     rank = list(specific.sort_values('Recall', ascending = False)['country_suffix']).index(country)
#     best_country_prompt = list(specific.sort_values('Recall', ascending = False)['country_suffix'][:5])
#     best_country_recall = list(round(specific.sort_values('Recall', ascending = False)['Recall'][:1], 3))
#     worst_country_prompt = list(specific.sort_values('Recall', ascending = False)['country_suffix'][-5:])
#     list_best_prompt.append(best_country_prompt)
#     list_worst_prompt.append(worst_country_prompt[::-1])
#     rank_list.append(rank+1)
    
#     bestest_country_prompt = best_country_prompt[0], best_country_recall
#     bestest_prompt.append(bestest_country_prompt) 
#     country_prompt_top_dict[country] = best_country_prompt
    
#     country_prompt_worst_dict[country] = worst_country_prompt[::-1]

In [18]:
#rank_list

In [62]:
#country_prompt_worst_dict

In [74]:
##### recall_incomelevel_list = []
country_income_level_country_prompt_recall_dict = {}

for country in unique_countries:
    all_levels = country_recall_df[country_recall_df['countries'] == country]
    all_levels_specific = all_levels[all_levels['country_suffix'] == 'poor country']
    incm_lst = ['poor', 'lower-middle', 'upper-middle', 'rich']
    recallforincm = []
    for level in incm_lst:
        if level in list(all_levels_specific['incomelevel']):
            recall_val = round(all_levels_specific[all_levels_specific['incomelevel'] == level]['recall'].item(), 3)
            recallforincm.append(recall_val)
        else: 
            recall_val = '-'
            recallforincm.append(recall_val)
    country_income_level_country_prompt_recall_dict[country] = recallforincm

country_income_level_country_prompt_recall_dict

{'Austria': ['-', '-', 0.181, 0.116],
 'Bangladesh': [0.428, '-', '-', '-'],
 'Bolivia': [0.295, 0.363, '-', '-'],
 'Brazil': ['-', 0.235, 0.249, 0.122],
 'Burkina Faso': [0.331, '-', '-', '-'],
 'Burundi': [0.384, '-', '-', '-'],
 'Cambodia': [0.412, 0.529, 0.479, '-'],
 'Cameroon': [0.339, 0.304, 0.339, '-'],
 'Canada': ['-', '-', '-', 0.354],
 'China': [0.394, 0.334, 0.271, 0.193],
 'Colombia': [0.391, 0.227, 0.221, 0.198],
 "Cote d'Ivoire": [0.37, '-', '-', '-'],
 'Czech Republic': ['-', '-', '-', 0.145],
 'Denmark': ['-', '-', '-', 0.135],
 'Egypt': ['-', '-', 0.338, '-'],
 'Ethiopia': ['-', 0.351, 0.345, 0.282],
 'France': ['-', '-', 0.245, 0.229],
 'Ghana': ['-', 0.444, '-', '-'],
 'Guatemala': ['-', 0.235, '-', '-'],
 'Haiti': [0.294, '-', '-', '-'],
 'India': [0.23, 0.302, 0.265, 0.259],
 'Indonesia': [0.444, 0.43, 0.337, 0.333],
 'Iran': ['-', '-', 0.276, 0.225],
 'Italy': ['-', '-', '-', 0.203],
 'Jordan': ['-', 0.329, '-', 0.195],
 'Kazakhstan': ['-', '-', 0.366, '-'],
 'Ke

In [75]:
for i in country_income_level_country_prompt_recall_dict.values():
    print(i[0])

-
0.428
0.295
-
0.331
0.384
0.412
0.339
-
0.394
0.391
0.37
-
-
-
-
-
-
-
0.294
0.23
0.444
-
-
-
-
0.41
-
-
0.398
0.293
-
-
0.404
0.42
-
0.444
0.273
0.36
0.323
0.343
0.346
-
-
0.451
-
0.296
0.267
-
-
-
-
-
0.387
0.267
0.357
0.284
-
-
-
-
-
0.304


In [22]:
## Query for overall recall across countries and topics for each country-suffix prompt
participating_countries = list(low_lvl_language_recall_df['countries'].unique())


overall_recall_per_prompt = {}

for country in participating_countries:
    sub_df = country_all_topic_all_country_perincomedf[country_all_topic_all_country_perincomedf['country_suffix'] == country]
    incm_lst = ['poor', 'lower-middle', 'upper-middle', 'rich']
    orecallforincm = []
    for level in incm_lst:
        orecall_val = round(sub_df[sub_df['Income level'] == level]['Recall'].item(), 3)
        orecallforincm.append(orecall_val)
    overall_recall_per_prompt[country] = orecallforincm

overall_recall_per_prompt

{'Bangladesh': [0.271, 0.319, 0.266, 0.221],
 'Bolivia': [0.3, 0.318, 0.262, 0.223],
 'Brazil': [0.254, 0.303, 0.323, 0.303],
 'Burkina Faso': [0.327, 0.303, 0.227, 0.185],
 'Burundi': [0.331, 0.279, 0.197, 0.154],
 'Cambodia': [0.289, 0.288, 0.213, 0.172],
 'Cameroon': [0.313, 0.289, 0.233, 0.192],
 'China': [0.213, 0.34, 0.369, 0.319],
 'Colombia': [0.3, 0.324, 0.275, 0.232],
 "Cote d'Ivoire": [0.23, 0.296, 0.325, 0.302],
 'Ethiopia': [0.334, 0.313, 0.269, 0.227],
 'Ghana': [0.314, 0.294, 0.267, 0.233],
 'Guatemala': [0.269, 0.314, 0.277, 0.233],
 'Haiti': [0.296, 0.331, 0.307, 0.269],
 'India': [0.239, 0.31, 0.306, 0.278],
 'Indonesia': [0.266, 0.328, 0.303, 0.266],
 'Jordan': [0.222, 0.308, 0.376, 0.371],
 'Kenya': [0.296, 0.318, 0.283, 0.236],
 'Liberia': [0.327, 0.303, 0.249, 0.205],
 'Malawi': [0.301, 0.32, 0.286, 0.254],
 'Mongolia': [0.259, 0.326, 0.308, 0.256],
 'Myanmar': [0.263, 0.304, 0.241, 0.195],
 'Nepal': [0.274, 0.307, 0.253, 0.213],
 'Nigeria': [0.294, 0.286, 0.256, 

In [23]:
for i in overall_recall_per_prompt.values():
    print(i[2])

0.266
0.262
0.323
0.227
0.197
0.213
0.233
0.369
0.275
0.325
0.269
0.267
0.277
0.307
0.306
0.303
0.376
0.283
0.249
0.286
0.308
0.241
0.253
0.256
0.321
0.361
0.266
0.317
0.337
0.249
0.378
0.243
0.302
0.345
0.257
0.293
0.283
0.314
0.372
0.421
0.33
0.285


In [24]:
sub_df

,Unnamed: 0,Income level,Forget ratio,Total images per income level,Forgotten images,country_suffix,Recognized images,Recall
152,0,poor,0.688281,11119,7653,Zimbabwe,3466,0.311719
153,1,lower-middle,0.689006,10997,7577,Zimbabwe,3420,0.310994
154,2,upper-middle,0.714922,11446,8183,Zimbabwe,3263,0.285078
155,3,rich,0.758089,12054,9138,Zimbabwe,2916,0.241911


In [25]:
unique_countries

['Austria',
 'Bangladesh',
 'Bolivia',
 'Brazil',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'China',
 'Colombia',
 "Cote d'Ivoire",
 'Czech Republic',
 'Denmark',
 'Egypt',
 'Ethiopia',
 'France',
 'Ghana',
 'Guatemala',
 'Haiti',
 'India',
 'Indonesia',
 'Iran',
 'Italy',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kyrgyzstan',
 'Lebanon',
 'Liberia',
 'Malawi',
 'Mexico',
 'Mongolia',
 'Myanmar',
 'Nepal',
 'Netherlands',
 'Nigeria',
 'Pakistan',
 'Palestine',
 'Papua New Guinea',
 'Peru',
 'Philippines',
 'Romania',
 'Russia',
 'Rwanda',
 'Serbia',
 'Somalia',
 'South Africa',
 'South Korea',
 'Spain',
 'Sri Lanka',
 'Sweden',
 'Switzerland',
 'Tanzania',
 'Thailand',
 'Togo',
 'Tunisia',
 'Turkey',
 'Ukraine',
 'United Kingdom',
 'United States',
 'Vietnam',
 'Zimbabwe']

In [26]:
poor = [round(0.224 - i[0], 2) for i in overall_recall_per_prompt.values()]
low_mid = [round(0.359 - i[1], 2) for i in overall_recall_per_prompt.values()]
up_mid = [round(0.456 - i[2], 2) for i in overall_recall_per_prompt.values()]
rich = [round(0.436 - i[3], 2) for i in overall_recall_per_prompt.values()]

In [27]:
round(sum(poor)/len(poor), 2)

-0.05

In [28]:
round(sum(low_mid)/len(low_mid), 2)

0.05

In [29]:
round(sum(up_mid)/len(up_mid), 2)

0.16

In [30]:
round(sum(rich)/len(rich), 2)

0.18

In [31]:
## all lower income per country

participating_countries = list(low_lvl_language_recall_df['countries'].unique())

In [42]:
len(participating_countries)

42

In [40]:
participating_countries

['Bangladesh',
 'Bolivia',
 'Brazil',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'China',
 'Colombia',
 "Cote d'Ivoire",
 'Ethiopia',
 'Ghana',
 'Guatemala',
 'Haiti',
 'India',
 'Indonesia',
 'Jordan',
 'Kenya',
 'Liberia',
 'Malawi',
 'Mongolia',
 'Myanmar',
 'Nepal',
 'Nigeria',
 'Pakistan',
 'Palestine',
 'Papua New Guinea',
 'Peru',
 'Philippines',
 'Rwanda',
 'Serbia',
 'Somalia',
 'South Africa',
 'South Korea',
 'Tanzania',
 'Thailand',
 'Togo',
 'Tunisia',
 'Ukraine',
 'United States',
 'Vietnam',
 'Zimbabwe']

In [ ]:
# Asia = 'Bangladesh','Cambodia','China','India','Indonesia','Jordan','Mongolia','Myanmar','Nepal','Pakistan','Palestine','Papua New Guinea',
# 'Philippines','South Korea','Thailand','Vietnam',
# America = 'Bolivia','Brazil','Colombia','Guatemala','Haiti','Peru','United States',
# Africa = 'Burkina Faso','Burundi','Cameroon',"Cote d'Ivoire",'Ethiopia','Kenya','Liberia','Malawi','Nigeria','Rwanda','Somalia','South Africa',
# 'Tanzania','Togo','Tunisia','Zimbabwe',
# Europe = 'Serbia','Ukraine'

In [10]:
participating_countries_rearr = ['Bangladesh','Cambodia','China','India','Indonesia','Jordan','Mongolia','Myanmar','Nepal','Pakistan','Palestine','Papua New Guinea',
'Philippines','South Korea','Thailand','Vietnam','Bolivia','Brazil','Colombia','Guatemala','Haiti','Peru','United States','Burkina Faso','Burundi','Cameroon',"Cote d'Ivoire",'Ethiopia','Ghana','Kenya','Liberia','Malawi','Nigeria','Rwanda','Somalia','South Africa',
'Tanzania','Togo','Tunisia','Zimbabwe','Serbia','Ukraine']

In [51]:
suff2recallbycountry = {}
list_oflists = []
for country in participating_countries_rearr:
    suff_recall = []
    for country_suff in participating_countries_rearr:
        specific = low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]
        recall = round(specific[specific['country_suffix'] == country_suff]['recall'].item(), 3)
        suff_recall.append(recall)
        
    suff2recallbycountry[country] = suff_recall
    list_oflists.append(suff_recall)

In [52]:
len(list_oflists)

42

In [57]:
suff2recallbycountry['Countries'] = participating_countries_rearr
    

In [58]:
len(suff2recallbycountry['Countries'])

42

In [59]:
import numpy as np

maplist = np.array(list_oflists)
np.save('country_suff_maplist.npy', maplist)

In [60]:
suff2recallbycountrydf = pd.DataFrame.from_dict(suff2recallbycountry)
suff_recallbycountrydf=suff2recallbycountrydf.set_index('Countries')
suff_recallbycountrydf

,Bangladesh,Cambodia,China,India,Indonesia,Jordan,Mongolia,Myanmar,Nepal,Pakistan,...,Nigeria,Rwanda,Somalia,South Africa,Tanzania,Togo,Tunisia,Zimbabwe,Serbia,Ukraine
Countries,,,,,,,,,,,,,,,,,,,,,
Bangladesh,0.467,0.427,0.295,0.403,0.397,0.310,0.155,0.402,0.416,0.411,...,0.296,0.279,0.167,0.160,0.328,0.259,0.205,0.102,0.172,0.165
Cambodia,0.438,0.477,0.313,0.281,0.409,0.215,0.196,0.467,0.362,0.222,...,0.408,0.361,0.130,0.134,0.272,0.297,0.191,0.131,0.143,0.165
China,0.284,0.394,0.467,0.250,0.389,0.382,0.381,0.308,0.310,0.254,...,0.234,0.272,0.037,0.179,0.269,0.233,0.256,0.122,0.298,0.371
India,0.393,0.346,0.253,0.431,0.335,0.313,0.216,0.259,0.420,0.405,...,0.275,0.259,0.185,0.163,0.319,0.269,0.211,0.104,0.215,0.227
Indonesia,0.404,0.429,0.305,0.349,0.437,0.297,0.278,0.397,0.347,0.319,...,0.323,0.313,0.167,0.160,0.350,0.308,0.234,0.116,0.197,0.237
Jordan,0.214,0.299,0.285,0.239,0.325,0.448,0.268,0.175,0.284,0.293,...,0.217,0.303,0.148,0.210,0.288,0.285,0.319,0.200,0.301,0.299
Mongolia,0.324,0.443,0.432,0.244,0.415,0.263,0.412,0.383,0.363,0.208,...,0.286,0.310,0.148,0.186,0.291,0.303,0.229,0.160,0.235,0.330
Myanmar,0.409,0.458,0.361,0.312,0.397,0.207,0.278,0.488,0.351,0.256,...,0.327,0.310,0.093,0.153,0.316,0.272,0.172,0.098,0.154,0.175
Nepal,0.437,0.410,0.333,0.383,0.378,0.220,0.216,0.370,0.466,0.339,...,0.282,0.316,0.185,0.135,0.368,0.272,0.194,0.122,0.184,0.216


In [61]:
suff_recallbycountrydf.to_csv('suff_recallbycountrydf.csv')

In [ ]:
# repeat for all income
suff2recallbycountry = {}
list_oflists = []
for country in participating_countries_rearr:
    suff_recall = []
    for country_suff in participating_countries_rearr:
        specific = country_recall_df[country_recall_df['countries'] == country]
        recall = round(specific[specific['country_suffix'] == country_suff]['recall'].item(), 3)
        suff_recall.append(recall)
        
    suff2recallbycountry[country] = suff_recall
    list_oflists.append(suff_recall)

## All income

In [11]:
suff2recallbycountry = {}
list_oflists = []
for country in participating_countries_rearr:
    suff_recall = []
    for country_suff in participating_countries_rearr:
        specific = avg_all_lvl_language_recall_df[avg_all_lvl_language_recall_df['country'] == country]
        recall = round(specific[specific['country suffix'] == country_suff]['recall'].item(), 3)
        suff_recall.append(recall)
        
    suff2recallbycountry[country] = suff_recall
    list_oflists.append(suff_recall)

In [12]:
len(list_oflists)

42

In [13]:
suff2recallbycountry['Countries'] = participating_countries_rearr

In [14]:
import numpy as np

maplist = np.array(list_oflists)
np.save('allcountry_suff_maplist.npy', maplist)

In [15]:
allsuff2recallbycountrydf = pd.DataFrame.from_dict(suff2recallbycountry)
allsuff2recallbycountrydf=allsuff2recallbycountrydf.set_index('Countries')
allsuff2recallbycountrydf

,Bangladesh,Cambodia,China,India,Indonesia,Jordan,Mongolia,Myanmar,Nepal,Pakistan,...,Nigeria,Rwanda,Somalia,South Africa,Tanzania,Togo,Tunisia,Zimbabwe,Serbia,Ukraine
Countries,,,,,,,,,,,,,,,,,,,,,
Bangladesh,0.467,0.438,0.274,0.441,0.397,0.280,0.155,0.402,0.415,0.443,...,0.281,0.279,0.167,0.168,0.322,0.259,0.205,0.102,0.155,0.140
Cambodia,0.438,0.473,0.270,0.283,0.400,0.196,0.196,0.467,0.341,0.260,...,0.333,0.361,0.130,0.128,0.267,0.297,0.191,0.131,0.127,0.162
China,0.284,0.413,0.500,0.335,0.401,0.359,0.381,0.308,0.330,0.350,...,0.272,0.272,0.037,0.214,0.294,0.233,0.256,0.122,0.302,0.310
India,0.393,0.363,0.242,0.497,0.336,0.300,0.216,0.259,0.442,0.446,...,0.281,0.259,0.185,0.178,0.296,0.269,0.211,0.104,0.215,0.212
Indonesia,0.404,0.453,0.324,0.389,0.443,0.286,0.278,0.397,0.359,0.388,...,0.309,0.313,0.167,0.186,0.341,0.308,0.234,0.116,0.201,0.222
Jordan,0.214,0.315,0.294,0.307,0.341,0.435,0.268,0.175,0.298,0.363,...,0.277,0.303,0.148,0.274,0.301,0.285,0.319,0.200,0.353,0.386
Mongolia,0.324,0.458,0.434,0.290,0.416,0.259,0.412,0.383,0.363,0.271,...,0.282,0.310,0.148,0.190,0.294,0.303,0.229,0.160,0.235,0.288
Myanmar,0.409,0.473,0.348,0.337,0.392,0.194,0.278,0.488,0.349,0.307,...,0.281,0.310,0.093,0.152,0.301,0.272,0.172,0.098,0.140,0.168
Nepal,0.437,0.416,0.306,0.415,0.369,0.204,0.216,0.370,0.457,0.375,...,0.265,0.316,0.185,0.140,0.324,0.272,0.194,0.122,0.154,0.170


In [16]:
allsuff2recallbycountrydf.to_csv('allsuff2recallbycountrydf.csv')